# Imports

In [1]:
from pandas_datareader import data as web
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import statistics
from statistics import mean
from datetime import date
from matplotlib.dates import DateFormatter
import numpy as np

import plotly.graph_objects as go
import plotly.figure_factory as ff

# Define Start and End date

In [2]:
# Dates should be in the format 'yyyy/mm/dd'
sdate = '2021/05/15'
edate = '2022/05/15'

# Get todays date
#today = date.today()
#print(today)

# Function that gets stock data and allows to save to a csv 

In [3]:
# Function that gets a dataframe by providing a ticker
def get_data_from_yahoo(ticker, save_csv = False, mas = [10,50,100], plot_price = True, plot_mas = False, plot_histogram = True):
    
    # Sets the time periods to be used
    start = dt.datetime(int(sdate.split("/")[0]), int(sdate.split("/")[1]), int(sdate.split("/")[2]))
    end = dt.datetime(int(edate.split("/")[0]), int(edate.split("/")[1]), int(edate.split("/")[2]))
    
    # Reads data into a dataframe
    df = web.DataReader(ticker, 'yahoo', start, end)
    
    
    #df.drop(columns = 'Adj Close')
    
    # Add daily return column
    df['Daily Return Pct'] = 100*(df['Adj Close'] / df['Adj Close'].shift(1) - 1)   
    
    # Create the simple moving averages
    for ma in mas:
        df['{} Moving Average'.format(ma)] = df['Adj Close'].rolling(window=ma).mean()
    
    # Round values to 2dp
    for column in df.columns:
        if column != 'Volume':
            df[column] = round(df[column], 2)
    
    # Saves dataframe into a csv if save_csv is True
    if save_csv:
        df.to_csv("/Users/harryfloyd/Desktop/" + ticker + '.csv')
        
    # Simple plotly plot of stock open and close
    if plot_price:
        
        graph = {
        'x': df.index,
        'open': df.Open,
        'close': df.Close,
        'high': df.High,
        'low': df.Low,
        'type': 'candlestick',
        'name': '{}'.format(ticker),
        'showlegend': True
        }

        layout = go.Figure(data = [graph], layout_title = "{}: Stock Price".format(ticker))

        # Change the background colour to black
        layout.update_layout(template = "plotly_dark")
        
        # Remove the slider on the bottom of the graph
        layout.update_layout(xaxis_rangeslider_visible=False)
        
        #layout.show()

    # Simple plot of stock moving averages 
    if plot_mas:
        for ma in mas:
            layout.add_trace(go.Scatter(x = df.index, y = df['{} Moving Average'.format(ma)], name = "{} MA".format(ma)))
            
    layout.show()    
    # Simple Plotly Histogram of stock daily_returns     
    if plot_histogram:
        
        fig = go.Figure(data=[go.Histogram(x=df['Daily Return Pct'])])
        
        fig.update_layout(
        title_text='Daily Return % Frequency Histogram', # title of plot
        xaxis_title_text='Daily Return %', # xaxis label
        yaxis_title_text='Count', # yaxis label
        bargap=0.1, # gap between bars of adjacent location coordinates
        )
        
        fig.show()
    #return df
    del df['Adj Close']
    df['Date'] = df.index
    
    fig = go.Figure(data=[go.Table(
        
        header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
        cells=dict(values=[df.Open, df.Close, df.Low, df.High, df.Volume, df['Daily Return Pct'], df['10 Moving Average'], df['50 Moving Average'], df['100 Moving Average'], df.Date,],
               fill_color='lavender',
               align='left'))
    ])
    
    fig.update_traces(header_font=dict(size = 10))
    fig.update_traces(cells_font=dict(size = 10))

    fig.show()

# Calculate how much of a stock can be bought with a given amount
### Calculate how much of a stock can be purchased at the start date given some amount.

In [4]:
def amount_of_stocks(tickers, amount):
    # Sets the time periods to be used
    start = dt.datetime(int(sdate.split("/")[0]), int(sdate.split("/")[1]), int(sdate.split("/")[2]))
    end = dt.datetime(int(edate.split("/")[0]), int(edate.split("/")[1]), int(edate.split("/")[2]))
    
    amounts = []
    
    # Reads data into a dataframe
    for ticker in tickers:
        df = web.DataReader(ticker, 'yahoo', start, end)
        amount_of_stock = round(amount / df['Open'][0], 1)
        print(f"{ticker}: {amount_of_stock}")
        amounts.append(amount_of_stock)
    return amounts

# Analyse a portfolio
### This function produces a chart of a given portfolio and also returns information about returns and standard deviation.  We also have the option to save the data of our portfolio as a csv and show the charts of the individual stocks that make up the portfolio.

In [5]:
# Function that gets a dataframe by providing a ticker, starting date and end date
def analyse_portfolio(tickers, amounts, save_csv = False, show_all_graphs = False):
    
    # Sets the time periods to be used
    start = dt.datetime(int(sdate.split("/")[0]), int(sdate.split("/")[1]), int(sdate.split("/")[2]))
    end = dt.datetime(int(edate.split("/")[0]), int(edate.split("/")[1]), int(edate.split("/")[2]))
    
    portfolio = pd.DataFrame()    
    
    # Reads data into a dataframe to create an empty dataframe
    portfolio['open_value'] = 0*web.DataReader('TSLA', 'yahoo', start, end).Open
    portfolio['close_value'] = 0*web.DataReader('TSLA', 'yahoo', start, end).Close
    portfolio['high_value'] = 0*web.DataReader('TSLA', 'yahoo', start, end).High
    portfolio['low_value'] = 0*web.DataReader('TSLA', 'yahoo', start, end).Low
    
    
    # Create portfolio values by adding columns from each stock
    index = 0
    for ticker in tickers:
         df = web.DataReader(ticker, 'yahoo', start, end)
            
         # Simple plot of stock open and close
         if show_all_graphs:
             
             graph = {
             'x': df.index,
             'open': df.Open,
             'close': df.Close,
             'high': df.High,
             'low': df.Low,
             'type': 'candlestick',
             'name': '{}'.format(ticker),
             'showlegend': True
             }   
             
             layout = go.Figure(data = [graph], layout_title = "{}: Stock Price".format(ticker))

             # Change the background colour to black
             layout.update_layout(template = "plotly_dark")
             layout.update_layout(xaxis_rangeslider_visible=False)
             layout.show()
        
         portfolio.open_value += df.Open * amounts[index]
         portfolio.close_value += df.Close * amounts[index]
         portfolio.high_value += df.High * amounts[index]
         portfolio.low_value += df.Low * amounts[index]
         index += 1
        
    # Add daily return column
    portfolio['daily_return_pct'] = 100*(portfolio.close_value / portfolio.close_value.shift(1) - 1) 
    
    # Round values to 2dp
    for column in portfolio.columns:
        portfolio[column] = round(portfolio[column], 2)
    
    # Saves dataframe into a csv if save_csv is True
    if save_csv:
        portfolio.to_csv("/Users/harryfloyd/Desktop/" + tickers + '.csv')
        
    # Simple plotly plot of portfolio open and close
    
    graph = {
    'x': portfolio.index,
    'open': portfolio.open_value,
    'close': portfolio.close_value,
    'high': portfolio.high_value,
    'low': portfolio.low_value,
    'type': 'candlestick',
    'name': 'Portfolio',
    'showlegend': True
    }
    
    layout = go.Figure(data = [graph], layout_title = "Portfolio Value")
    
    # Change the background colour to black
    layout.update_layout(template = "plotly_dark")
    layout.update_layout(xaxis_rangeslider_visible=False)
    layout.show()
    
    # Max daily increase and decrease
    max_percent_increase = portfolio.daily_return_pct.max()
    max_percent_decrease = portfolio.daily_return_pct.min()
    
    print(f"Max daily percent increase: {max_percent_increase}%, Max daily percent decrease: {max_percent_decrease}%")
    
    # Total return over timeframe
    return_over_time = round(100*(portfolio.close_value[-1] / portfolio.open_value[0] - 1), 2)
    
    print(f"Total return over timeframe: {return_over_time}%")
    
    # This is a hack to the get the mean daily return
    # The first value is 'Nan' and so gives the value 'Nan'
    # This replaces 'Nan with the first daily_return_pct'
    portfolio.daily_return_pct[0] = portfolio.daily_return_pct[1]
    # Mean daily return
    mean_daily_return = round(mean(portfolio.daily_return_pct), 2)
    
    print(f"Mean daily return: {mean_daily_return}%")
    
    stddev_of_returns = round(statistics.stdev(portfolio.daily_return_pct), 2)
    print(f"Standard Deviation of returns: {stddev_of_returns}%")
    
    # Max percent increase and decrease from start value
    highest_value = portfolio.close_value.max()
    portfolio_high = round(100*(highest_value / portfolio.close_value[0]  - 1), 2)
    print(f"Largest increase from start value: {portfolio_high}%")
    
    lowest_value = portfolio.close_value.min()
    portfolio_low = round(100*(lowest_value / portfolio.close_value[0]  - 1), 2)
    print(f"Largest decrease from start value: {portfolio_low}%")
        
    #print (portfolio)
    #return layout

# Function that returns a Dataframe from a csv
### Open a saved csv file as a dataframe

In [6]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv("/Users/harryfloyd/Desktop/" + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

# Function to plot stock daily returns
### A simple plot of stock daily returns. To be used with get_data_from_yahoo to obtain df.

In [7]:
def plot_pct_change(df, ticker):
    df.daily_return_pct.plot(label='{} Pct Change'.format(ticker))
    plt.hlines(0, sdate, edate,  color='red')
    plt.legend()
    plt.title('{} Pct Change'.format(ticker))
    plt.ylabel('Pct Change')
    plt.show()

# Get the correlation between stocks

In [8]:
def correlation(tickers, show_all_graphs = False):
    # Sets the time periods to be used
    start = dt.datetime(int(sdate.split("/")[0]), int(sdate.split("/")[1]), int(sdate.split("/")[2]))
    end = dt.datetime(int(edate.split("/")[0]), int(edate.split("/")[1]), int(edate.split("/")[2]))
    
    # Reads data into a dataframe
    dfcomp = web.DataReader(tickers,'yahoo', start, end)['Adj Close']
    # Get percent changes of adjusted close to find the returns
    retscomp = dfcomp.pct_change()
    # Get the correlation of the returns
    corr = retscomp.corr()
    
    index = 0
    if show_all_graphs:
        for ticker in tickers:
             ticker = web.DataReader(ticker, 'yahoo', start, end) 
                
             graph = {
             'x': ticker.index,
             'open': ticker.Open,
             'close': ticker.Close,
             'high': ticker.High,
             'low': ticker.Low,
             'type': 'candlestick',
             'name': '{}'.format(tickers[index]),
             'showlegend': True
             }

             layout = go.Figure(data = [graph], layout_title = "{}: Stock Price".format(tickers[index]))

             # Change the background colour to black
             layout.update_layout(template = "plotly_dark")
                
             layout.update_layout(xaxis_rangeslider_visible=False)

             layout.show()
             index += 1
    
    # Plotly Correlation heat map
    x = list(corr.columns)
    y = list(corr.index)
    z = np.array(corr)
    
    fig = ff.create_annotated_heatmap(
    z,
    x = x,
    y = y,
    annotation_text = np.around(np.array(corr), decimals=2),
    hoverinfo='z',
    colorscale='Sunset'
    )
    
    fig.show()

# Find the total return of a list of stocks over the defined period

In [9]:
def total_return(tickers):
    start = dt.datetime(int(sdate.split("/")[0]), int(sdate.split("/")[1]), int(sdate.split("/")[2]))
    end = dt.datetime(int(edate.split("/")[0]), int(edate.split("/")[1]), int(edate.split("/")[2]))
    
    # Reads data into a dataframe
    dfcomp = web.DataReader(tickers,'yahoo', start, end)['Adj Close']
    
    for i in (tickers):
        return_over_time = round(100*(dfcomp[i][-1] / dfcomp[i][0] - 1), 2)
        print(f"{i} return over timeframe: {return_over_time}%")

# Find the Beta of a stock

In [10]:
def beta(tickers, save_csv = False):
    # Sets the time periods to be used
    start = dt.datetime(int(sdate.split("/")[0]), int(sdate.split("/")[1]), int(sdate.split("/")[2]))
    end = dt.datetime(int(edate.split("/")[0]), int(edate.split("/")[1]), int(edate.split("/")[2]))
    
    # Reads data into a dataframe
    dfcomp = web.DataReader(tickers,'yahoo', start, end)['Adj Close']
    
    
    index = 0
    for ticker in tickers:
         ticker = web.DataReader(ticker, 'yahoo', start, end)
            
         # Simple plot of stock open and close
         ticker.Open.plot(label='Daily Open')
         ticker.Close.plot(label='Daily Close')
         plt.legend()
         plt.title(f"{tickers[index]}: Stock Price")
         plt.ylabel('Stock Price')
         plt.show() 
         index += 1           
            
    # Get percent changes of adjusted close to find the returns
    retscomp = dfcomp.pct_change()
    
    # remove row 1 as it contains Nan values
    retscomp = retscomp.dropna(axis=0)
    print(retscomp)
    
    # Get the correlation matix of returns
    # A value of 1 means the asstes are perfectly correlated
    # A value of 0 means the asstes are uncorrelated
    # A value of -1 means the stocks are perfectly uncorrelated
    corr = retscomp.corr()
    print(corr)
    
    # Use np.cov(a,b) to get the covariance between column 1 which is the stock returns
    # and column 2 which is the index returns
    # np.cov() calculates the sample covarince by default
    # To calculate the population variance, use bias = True
    covariance = np.cov(retscomp.iloc[:,0], retscomp.iloc[:,1], bias = True)
    
    # Calculate the variance and the sd of the index and the stock
    # np.var() calculates the population variance
    variance_of_index = np.var(retscomp.iloc[:,1])
    sd_of_index = variance_of_index**0.5
    
    variance_of_stock = np.var(retscomp.iloc[:,0])
    sd_of_stock = variance_of_stock**0.5
    
    beta = covariance[0][1] / variance_of_index
    # we can also use the correlation and standard deviations to calculate beta
    # beta = corr[tickers[0]][tickers[1]] * (sd_of_stock/sd_of_index)

    
    # Save as a csv
    if save_csv:
        retscomp.to_csv("/Users/harryfloyd/Desktop/" + tickers[0] + '&' + tickers[1] + '.csv')
    
    print(f"Beta of {tickers[0]}: {beta}") 

# Create a candlestick stock chart

### This is a demo function that requires some editing. The package being used (plotly) seems to be incredibly useful

In [11]:
def candlestick_plot(ticker, mas = [10, 50, 100], show_mas = False):
    # Sets the time periods to be used
    start = dt.datetime(int(sdate.split("/")[0]), int(sdate.split("/")[1]), int(sdate.split("/")[2]))
    end = dt.datetime(int(edate.split("/")[0]), int(edate.split("/")[1]), int(edate.split("/")[2]))
    
    # Reads data into a dataframe
    df = web.DataReader(ticker,'yahoo', start, end)
    
    # Create the simple moving averages
    for ma in mas:
        df['{} Moving Average'.format(ma)] = df['Close'].rolling(window=ma).mean()
    
    graph = {
    'x': df.index,
    'open': df.Open,
    'close': df.Close,
    'high': df.High,
    'low': df.Low,
    'type': 'candlestick',
    'name': '{}'.format(ticker),
    'showlegend': True
    }
    
    layout = go.Figure(data = [graph], layout_title = "{} Stock".format(ticker))
    
    if show_mas:
        for ma in mas:
            layout.add_trace(go.Scatter(x = df.index, y = df['{} Moving Average'.format(ma)], name = "{} MA".format(ma)))
            
    
    # Change the background colour to black
    layout.update_layout(template = "plotly_dark")
    
    return layout
    